In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

os.listdir("/content/drive/MyDrive")


['Colab Notebooks',
 'Getting started.pdf',
 'Book1.csv',
 'AlzheimersData.csv',
 'Churn_Modelling.csv',
 'customers-1000.csv',
 'Daisi_Williams_Academic_grades.pdf',
 'Daisi_Williams_Curriculum_Vitae.pdf',
 '6439.jpg',
 'send_Williams.zip',
 'UIQC Membership Form – DAISI WILLIAMS.gdoc',
 '2025-10',
 'Vids Exports',
 'Untitled video (3).gvid',
 'Untitled video (2).gvid',
 'Untitled video (1).gvid',
 'Untitled video.gvid',
 'Daisi Williams_Hackathon_Solution.pdf',
 'explorer_KxO7TN12bp.mp4',
 'merged_with_CM_5dp.csv',
 'samplequestions.pdf',
 'Experimental Lab 4 - Waldron.docx',
 'Daisi Williams SOP.pdf',
 'Determination_of_Plancks_Constant_by_Measurement_of_the_Photoelectric_Effect.pdf',
 'Personal Statement MSE.gdoc',
 'Personal Statement Physics.gdoc',
 'SOP phyiscs.gdoc',
 'SOP MSE.gdoc',
 'weather_cnn_rnn_model.h5',
 'class_names.json',
 'cleaned_merged_with_CM_5dp.csv']

In [ ]:
import pandas as pd
import numpy as np

CSV_PATH = "/content/drive/MyDrive/merged_with_CM_5dp.csv"
df = pd.read_csv(CSV_PATH)

bad_rows = [4654, 21291]   # <-- pandas row indices (0-based)

df_clean = df.drop(index=bad_rows).reset_index(drop=True)

print("Original rows:", len(df))
print("Cleaned rows :", len(df_clean))
print("Dropped rows :", bad_rows)


Original rows: 21323
Cleaned rows : 21321
Dropped rows : [4654, 21291]


In [ ]:
baseline_E1 = df_clean["E1-DFTB"].to_numpy(dtype=float)

print("E1-DFTB range after cleaning:",
      baseline_E1.min(), baseline_E1.max())

# Optional: confirm no huge outliers remain
print("Count E1-DFTB > 10:", np.sum(baseline_E1 > 10))


E1-DFTB range after cleaning: 0.0004 21011.0
Count E1-DFTB > 10: 25


In [ ]:
# Rows with suspiciously large values
mask_large = df_clean["E1-DFTB"] >= 2.0

# Rows with near-zero values
mask_small = df_clean["E1-DFTB"] < 0.01

print("Count E1-DFTB >= 2.0:", mask_large.sum())
print("Count E1-DFTB < 0.01:", mask_small.sum())

df_clean.loc[mask_large | mask_small,
             ["gdb9_index", "E1-DFTB", "E1-CC2"]].head(10)


Count E1-DFTB >= 2.0: 25
Count E1-DFTB < 0.01: 4


,gdb9_index,E1-DFTB,E1-CC2
201,207,207.0,0.21334
1381,1421,1421.0,0.16026
1434,1474,1474.0,0.23437
1980,2036,2036.0,0.27893
2375,2434,2434.0,0.25141
2503,2564,2564.0,0.16484
2572,2633,2633.0,0.21365
3178,3239,3239.0,0.30893
3333,3394,3394.0,0.16587
3934,4012,4012.0,0.22477


In [ ]:
mask_physical = (
    (df_clean["E1-DFTB"] >= 0.01) &
    (df_clean["E1-DFTB"] <= 2.0)
)

df_phys = df_clean.loc[mask_physical].reset_index(drop=True)

print("Rows before:", len(df_clean))
print("Rows after :", len(df_phys))
print("Dropped    :", len(df_clean) - len(df_phys))


Rows before: 21321
Rows after : 21292
Dropped    : 29


In [ ]:
baseline_E1 = df_phys["E1-DFTB"].to_numpy(dtype=float)

print("E1-DFTB range after filtering:",
      baseline_E1.min(), baseline_E1.max())

print("Count E1-DFTB > 2.0:", (baseline_E1 > 2.0).sum())
print("Count E1-DFTB < 0.01:", (baseline_E1 < 0.01).sum())


E1-DFTB range after filtering: 0.01352 0.84876
Count E1-DFTB > 2.0: 0
Count E1-DFTB < 0.01: 0


In [ ]:
cm_cols = [c for c in df_phys.columns if c.startswith("CM_")]

X = df_phys[cm_cols].to_numpy(dtype=float)
baseline_E1 = df_phys["E1-DFTB"].to_numpy(dtype=float)
target_E1   = df_phys["E1-CC2"].to_numpy(dtype=float)

dE1 = target_E1 - baseline_E1

print("X shape:", X.shape)
print("ΔE1 range:", dE1.min(), dE1.max())
print("ΔE1 mean ± std:", dE1.mean(), dE1.std())

assert np.isfinite(X).all()
assert np.isfinite(dE1).all()


X shape: (21292, 26)
ΔE1 range: -0.62257 0.30557999999999996
ΔE1 mean ± std: -0.015456138925417997 0.09605581848452671


In [ ]:
import numpy as np
import pandas as pd

print("===== DATASET SANITY CHECK =====")

# -----------------------------
# 1. Shape & basic integrity
# -----------------------------
print("\n[1] Dataset shape")
print("Rows:", len(df_phys))
print("Columns:", df_phys.shape[1])

# -----------------------------
# 2. Required columns present
# -----------------------------
required_cols = ["E1-DFTB", "E1-CC2"]
cm_cols = [c for c in df_phys.columns if c.startswith("CM_")]

print("\n[2] Required columns check")
print("E1-DFTB present:", "E1-DFTB" in df_phys.columns)
print("E1-CC2 present :", "E1-CC2" in df_phys.columns)
print("Number of CM features:", len(cm_cols))

# -----------------------------
# 3. No NaN / Inf anywhere
# -----------------------------
print("\n[3] NaN / Inf check")
print("NaNs total:", df_phys.isna().sum().sum())
print("Infs total:",
      np.isinf(df_phys.select_dtypes(include=[float])).sum().sum())

# -----------------------------
# 4. Physical ranges (energies)
# -----------------------------
print("\n[4] Physical range check (Hartree)")
print("E1-DFTB range:",
      df_phys["E1-DFTB"].min(),
      df_phys["E1-DFTB"].max())

print("E1-CC2 range:",
      df_phys["E1-CC2"].min(),
      df_phys["E1-CC2"].max())

# -----------------------------
# 5. Δ-learning target check
# -----------------------------
dE1 = df_phys["E1-CC2"] - df_phys["E1-DFTB"]

print("\n[5] ΔE1 statistics")
print("ΔE1 min / max:", dE1.min(), dE1.max())
print("ΔE1 mean ± std:", dE1.mean(), dE1.std())

# -----------------------------
# 6. Feature matrix integrity
# -----------------------------
X = df_phys[cm_cols].to_numpy(dtype=float)

print("\n[6] Feature matrix check")
print("X shape:", X.shape)
print("X finite:", np.isfinite(X).all())

# -----------------------------
# 7. Correlation sanity (optional but powerful)
# -----------------------------
corr = np.corrcoef(df_phys["E1-DFTB"], df_phys["E1-CC2"])[0, 1]
print("\n[7] Baseline correlation")
print("Corr(E1-DFTB, E1-CC2):", corr)

print("\n===== SANITY CHECK PASSED =====")


===== DATASET SANITY CHECK =====

[1] Dataset shape
Rows: 21292
Columns: 35

[2] Required columns check
E1-DFTB present: True
E1-CC2 present : True
Number of CM features: 26

[3] NaN / Inf check
NaNs total: 0
Infs total: 0

[4] Physical range check (Hartree)
E1-DFTB range: 0.01352 0.84876
E1-CC2 range: 0.06957 0.51384

[5] ΔE1 statistics
ΔE1 min / max: -0.62257 0.30557999999999996
ΔE1 mean ± std: -0.015456138925417997 0.09605807424260773

[6] Feature matrix check
X shape: (21292, 26)
X finite: True

[7] Baseline correlation
Corr(E1-DFTB, E1-CC2): 0.09016857080034928

===== SANITY CHECK PASSED =====


In [ ]:
DRIVE_OUT = "/content/drive/MyDrive/cleaned_merged_with_CM_5dp.csv"
df_phys.to_csv(DRIVE_OUT, index=False)

print("Saved to Google Drive at:", DRIVE_OUT)


Saved to Google Drive at: /content/drive/MyDrive/cleaned_merged_with_CM_5dp.csv


In [ ]:
import pandas as pd

CSV_PATH = "/content/drive/MyDrive/cleaned_merged_with_CM_5dp.csv"
df = pd.read_csv(CSV_PATH)


In [ ]:
cm_cols = [c for c in df.columns if c.startswith("CM_")]
X = df[cm_cols].to_numpy(dtype=float)

baseline_E1 = df["E1-DFTB"].to_numpy(dtype=float)
target_E1   = df["E1-CC2"].to_numpy(dtype=float)
dE1 = target_E1 - baseline_E1


In [ ]:
print("Rows:", len(df))
print("E1-DFTB range:", df["E1-DFTB"].min(), df["E1-DFTB"].max())


Rows: 21292
E1-DFTB range: 0.01352 0.84876


In [ ]:
!pip -q install tensorflow

import numpy as np
import pandas as pd


In [ ]:
CSV_PATH = "/content/drive/MyDrive/cleaned_merged_with_CM_5dp.csv"
df = pd.read_csv(CSV_PATH)

cm_cols = [c for c in df.columns if c.startswith("CM_")]
X = df[cm_cols].to_numpy(dtype=np.float32)

baseline_E1 = df["E1-DFTB"].to_numpy(dtype=np.float32)
target_E1   = df["E1-CC2"].to_numpy(dtype=np.float32)
y = (target_E1 - baseline_E1).astype(np.float32)   # ΔE1

print("X shape:", X.shape)
print("y (ΔE1) mean ± std:", y.mean(), y.std())
print("E1-DFTB range:", baseline_E1.min(), baseline_E1.max())
print("E1-CC2  range:", target_E1.min(), target_E1.max())

assert np.isfinite(X).all()
assert np.isfinite(y).all()


X shape: (21292, 26)
y (ΔE1) mean ± std: -0.015456138 0.09605582
E1-DFTB range: 0.01352 0.84876
E1-CC2  range: 0.06957 0.51384


In [ ]:
from sklearn.model_selection import train_test_split

idx = np.arange(X.shape[0])

idx_train, idx_tmp = train_test_split(idx, test_size=0.30, random_state=42)
idx_val, idx_test  = train_test_split(idx_tmp, test_size=0.50, random_state=42)

X_train, X_val, X_test = X[idx_train], X[idx_val], X[idx_test]
y_train, y_val, y_test = y[idx_train], y[idx_val], y[idx_test]

E1_DFTB_test = baseline_E1[idx_test]
E1_CC2_test  = target_E1[idx_test]

print("Train/Val/Test:", len(idx_train), len(idx_val), len(idx_test))


Train/Val/Test: 14904 3194 3194


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

print("Scaled feature mean (train):", X_train_s.mean().round(4))
print("Scaled feature std  (train):", X_train_s.std().round(4))


Scaled feature mean (train): -0.0
Scaled feature std  (train): 1.0


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

tf.keras.utils.set_random_seed(42)

model = models.Sequential([
    layers.Input(shape=(X_train_s.shape[1],)),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.10),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.10),
    layers.Dense(32, activation="relu"),
    layers.Dense(1)   # predicts ΔE1
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="mae",
    metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         3,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,825 (54.00 KB)

 Trainable params: 13,825 (54.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=20,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=8,
        min_lr=1e-6
    )
]

history = model.fit(
    X_train_s, y_train,
    validation_data=(X_val_s, y_val),
    epochs=300,
    batch_size=256,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/300
59/59 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - loss: 0.1992 - rmse: 0.3025 - val_loss: 0.0630 - val_rmse: 0.0814 - learning_rate: 0.0010
Epoch 2/300
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0690 - rmse: 0.0903 - val_loss: 0.0579 - val_rmse: 0.0747 - learning_rate: 0.0010
Epoch 3/300
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0624 - rmse: 0.0807 - val_loss: 0.0554 - val_rmse: 0.0713 - learning_rate: 0.0010
Epoch 4/300
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0589 - rmse: 0.0758 - val_loss: 0.0545 - val_rmse: 0.0704 - learning_rate: 0.0010
Epoch 5/300
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0581 - rmse: 0.0749 - val_loss: 0.0537 - val_rmse: 0.0690 - learning_rate: 0.0010
Epoch 6/300
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0563 - rmse: 0.0720 - val_loss: 0.0523 - val_rmse: 0.0675 - learning_rate: 0.0010
Epoch 7/300
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0550 - rmse: 0.0710 - val_loss: 0.0525 - val_rmse: 0.0676 - learning_rate: 0.001

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Predict ΔE1
dE_pred = model.predict(X_test_s, verbose=0).reshape(-1)

# Corrected excitation energy
E1_pred = E1_DFTB_test + dE_pred

# Baseline metrics
mae_base  = mean_absolute_error(E1_CC2_test, E1_DFTB_test)
rmse_base = np.sqrt(mean_squared_error(E1_CC2_test, E1_DFTB_test))

# Corrected metrics
mae_corr  = mean_absolute_error(E1_CC2_test, E1_pred)
rmse_corr = np.sqrt(mean_squared_error(E1_CC2_test, E1_pred))
r2_corr   = r2_score(E1_CC2_test, E1_pred)

print("Baseline MAE (DFTB → CC2):", mae_base)
print("Corrected MAE (DFTB + Δ → CC2):", mae_corr)
print("Baseline RMSE:", rmse_base)
print("Corrected RMSE:", rmse_corr)
print("R² (corrected):", r2_corr)
print("MAE improvement factor:", mae_base / mae_corr)


Baseline MAE (DFTB → CC2): 0.07434999942779541
Corrected MAE (DFTB + Δ → CC2): 0.04172660782933235
Baseline RMSE: 0.09703889737576989
Corrected RMSE: 0.05344146842749379
R² (corrected): -0.5310747623443604
MAE improvement factor: 1.781836657604598


In [ ]:
abs_err = np.abs(E1_CC2_test - E1_pred)

print("P50 AE (median):", np.percentile(abs_err, 50))
print("P90 AE:", np.percentile(abs_err, 90))
print("P95 AE:", np.percentile(abs_err, 95))
print("Max AE:", abs_err.max())


P50 AE (median): 0.03397943
P90 AE: 0.088039145
P95 AE: 0.10634223
Max AE: 0.23748454


In [ ]:
import pandas as pd

comparison_df = pd.DataFrame([
    # ---------------- PAPER RESULTS ----------------
    {
        "Source": "Ramakrishnan et al. (JCTC 2015)",
        "Baseline": "TDDFT",
        "ML model": "Kernel Ridge (Laplacian)",
        "Descriptor": "Coulomb Matrix",
        "Training size": "5k",
        "MAE (eV)": 0.13,
        "RMSE (eV)": None,
        "P50 AE (eV)": None,
        "P90 AE (eV)": None,
        "P95 AE (eV)": None,
        "Max AE (eV)": None,
    },
    {
        "Source": "Ramakrishnan et al. (JCTC 2015)",
        "Baseline": "TDDFT",
        "ML model": "Kernel Ridge (Laplacian)",
        "Descriptor": "Bag of Bonds",
        "Training size": "5k",
        "MAE (eV)": 0.09,
        "RMSE (eV)": None,
        "P50 AE (eV)": None,
        "P90 AE (eV)": None,
        "P95 AE (eV)": None,
        "Max AE (eV)": None,
    },

    # ---------------- YOUR BASELINE ----------------
    {
        "Source": "This work",
        "Baseline": "DFTB",
        "ML model": "None (baseline)",
        "Descriptor": "—",
        "Training size": "—",
        "MAE (eV)": mae_base,
        "RMSE (eV)": rmse_base,
        "P50 AE (eV)": np.percentile(np.abs(E1_CC2_test - E1_DFTB_test), 50),
        "P90 AE (eV)": np.percentile(np.abs(E1_CC2_test - E1_DFTB_test), 90),
        "P95 AE (eV)": np.percentile(np.abs(E1_CC2_test - E1_DFTB_test), 95),
        "Max AE (eV)": np.abs(E1_CC2_test - E1_DFTB_test).max(),
    },

    # ---------------- YOUR ML RESULT ----------------
    {
        "Source": "This work",
        "Baseline": "DFTB",
        "ML model": "Neural Network (Δ-learning)",
        "Descriptor": "Reduced Coulomb Matrix",
        "Training size": "15k+",
        "MAE (eV)": mae_corr,
        "RMSE (eV)": rmse_corr,
        "P50 AE (eV)": np.percentile(abs_err, 50),
        "P90 AE (eV)": np.percentile(abs_err, 90),
        "P95 AE (eV)": np.percentile(abs_err, 95),
        "Max AE (eV)": abs_err.max(),
    }
])

comparison_df


,Source,Baseline,ML model,Descriptor,Training size,MAE (eV),RMSE (eV),P50 AE (eV),P90 AE (eV),P95 AE (eV),Max AE (eV)
0,Ramakrishnan et al. (JCTC 2015),TDDFT,Kernel Ridge (Laplacian),Coulomb Matrix,5k,0.130000,NaN,NaN,NaN,NaN,NaN
1,Ramakrishnan et al. (JCTC 2015),TDDFT,Kernel Ridge (Laplacian),Bag of Bonds,5k,0.090000,NaN,NaN,NaN,NaN,NaN
2,This work,DFTB,None (baseline),—,—,0.074350,0.097039,0.061100,0.161387,0.202843,0.367750
3,This work,DFTB,Neural Network (Δ-learning),Reduced Coulomb Matrix,15k+,0.041727,0.053441,0.033979,0.088039,0.106342,0.237485
